# Exploration notebook
This notebook is used for ad-hoc exploration of data

## Setup
Run cells in this section to get your environment setup

In [ ]:
# Getting data from an existing table to just prove out connectivity
from databricks.connect.session import DatabricksSession

spark = DatabricksSession.builder.profile("cory").getOrCreate()
df = spark.sql("select * from object_computing.parametric_insurance.hurricane_data limit 10")
df.show(5)

: 

: 